# Ensemble

In [1]:
#!pip install ensemble-boxes

In [1]:
from ensemble_boxes import *
import pandas as pd
import csv
from tqdm import tqdm

In [2]:
target1_path = "./ensemble_target/target1.csv"
target2_path = "./ensemble_target/target2.csv"

target1 = pd.read_csv(target1_path, na_values=['nan'])['PredictionString'].values
target2 = pd.read_csv(target2_path, na_values=['nan'])['PredictionString'].values


In [3]:
print(len(target1), len(target2))

assert len(target1) == len(target2), "target file has difference length."

4871 4871


In [4]:
ensemble = []

In [5]:
normalize = 1024.
for i in range (len(target1)): # 이미지 한 장마다 ensemble 시도
    boxes_list = [[] for _ in range(2)]
    scores_list = [[] for _ in range(2)]
    labels_list = [[] for _ in range(2)]
    
    if type(target1[i]) == float:
        prediction_string = target2[i]
        print("target1 has None ", i)
        
    elif type(target2[i]) == float:
        prediction_string = target1[i]
        print("target2 has None ", i)
        
    elif target1[i] == None and target2[i] == None:
        prediction_string = ""
        print("each target is None")
        
    else:
        pred_1 = target1[i].split()
        pred_2 = target2[i].split()

        assert len(pred_1) % 6 == 0
        assert len(pred_2) % 6 == 0

        # parse targe1
        for k in range(0, len(pred_1), 6):

            class_num = int(pred_1[k])
            score = float(pred_1[k+1])
            boxes = list(map(float, ([pred_1[k+2], pred_1[k+3], pred_1[k+4], pred_1[k+5]])))
            for j in range(len(boxes)):
                boxes[j] /= normalize
            labels_list[0].append(class_num)
            scores_list[0].append(score)
            boxes_list[0].append(boxes)

        # parse target2
        for k in range(0, len(pred_2), 6):

            class_num = int(pred_2[k])
            score = float(pred_2[k+1])
            boxes = list(map(float,([pred_2[k+2], pred_2[k+3], pred_2[k+4], pred_2[k+5]])))
            for j in range(len(boxes)):
                boxes[j] /= normalize
            labels_list[1].append(class_num)
            scores_list[1].append(score)
            boxes_list[1].append(boxes)

        weights = [1, 1]
        iou_thr = 0.7
        skip_box_thr = 0.0001
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
#         snms_boxes, snms_scores, snms_labels = soft_nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, sigma=0.1, thresh=skip_box_thr)
        
        prediction_string = []
        for label, score, box in zip(labels, scores, boxes):
            bbox = ' '.join([str(i*normalize) for i in box])
            to_string = ' '.join([str(int(label)), str(score), bbox])
            prediction_string.append(to_string)
        
        prediction_string = ' '.join(prediction_string)
    
    ensemble.append({"PredictionString":prediction_string, "image_id":f"test/{i:04d}.jpg"})
    # ensemble["PredictionString"].append(prediction_string)
    # ensemble["image_id"].append(f"test/{i:04d}")


target1 has None  1474
target1 has None  2150
target1 has None  3266
target2 has None  3389
target1 has None  4200
target1 has None  4787


In [6]:
with open('ensemble.csv','w') as f:
    w = csv.DictWriter(f,  fieldnames=["PredictionString","image_id"])
    w.writeheader()
    w.writerows(ensemble)